In [62]:
# Libraries for downloading data from FTP
import shutil
import urllib.request as req
from contextlib import closing

# Library for uploading data to S3
import boto3

# Libraries for handling data
import rasterio as rio
import pandas as pd
import numpy as np
import scipy

# Libraries for various helper functions
from datetime import datetime
import os
import threading
import sys
from glob import glob
from netCDF4 import Dataset

cli.015 http://edgar.jrc.ec.europa.eu/methodology.php
SRC: ftp://cidportal.jrc.ec.europa.eu/jrc-opendata/EDGAR/datasets/v431_v2/
Have data for BC, CO, NH3, NMVOC_bio, NMVOC_fossil, NOx, OC, PM2.5, PM10, SO2
These are broken down by (may be more) AGR, ENE, IND, OTH, PPA, PRO, RCO, REF, SWD, TNR_Aviation_CDS, TNR_Aviation_CRS, TNR_Aviation_LTO, TNR_Aviation_SPS, TNR_Other, TNR_Ship, TOTALS, TRF, TRO
File type: netcdf

In [105]:
# Directions for how to read contents of an FTP directory
# https://stackoverflow.com/questions/2289768/python-and-urllib
remote_path = "ftp://cidportal.jrc.ec.europa.eu/jrc-opendata/EDGAR/datasets/v431_v2"
remote_path_BC = remote_path + "/BC/TOTALS/"
file = req.urlopen(remote_path_BC).read().splitlines()
[name.split()[-1] for name in file]

[b'copyright.txt',
 b'v431_v2_REFERENCE_BC_1970-2010_nc.zip',
 b'v431_v2_REFERENCE_BC_1970-2010_txt.zip',
 b'v431_v2_REFERENCE_BC_1970.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1970.zip',
 b'v431_v2_REFERENCE_BC_1971.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1971.zip',
 b'v431_v2_REFERENCE_BC_1972.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1972.zip',
 b'v431_v2_REFERENCE_BC_1973.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1973.zip',
 b'v431_v2_REFERENCE_BC_1974.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1974.zip',
 b'v431_v2_REFERENCE_BC_1975.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1975.zip',
 b'v431_v2_REFERENCE_BC_1976.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1976.zip',
 b'v431_v2_REFERENCE_BC_1977.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1977.zip',
 b'v431_v2_REFERENCE_BC_1978.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1978.zip',
 b'v431_v2_REFERENCE_BC_1979.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1979.zip',
 b'v431_v2_REFERENCE_BC_1980.0.1x0.1.zip',
 b'v431_v2_REFERENCE_BC_1980.zip',
 b'v431_v2_REFERENCE_BC_1981.0.1x0.1

In [ ]:
# Files are zipped... will need to download and unzip

In [3]:
particular = "v431_v2_REFERENCE_BC_1970-2010_nc.zip"
dl_path = remote_path_BC + particular

local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/"
file_name = "nc_test.zip"
local_file = local_folder + file_name

with closing(req.urlopen(dl_path)) as r:
    with open(local_file, 'wb') as f:
        shutil.copyfileobj(r, f)

In [107]:
particulars = ['v431_v2_REFERENCE_BC_2010.0.1x0.1.zip','v431_v2_REFERENCE_BC_2010.zip']
dl_path = [remote_path_BC + particular for particular in particulars]

local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/"

for ix, path in enumerate(dl_path):
    file_name = particulars[ix]
    local_file = local_folder + file_name
    with closing(req.urlopen(path)) as r:
        with open(local_file, 'wb') as f:
             shutil.copyfileobj(r, f)


In [115]:
glob(local_folder + "nc_test/*")

['/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/_readme.txt',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1970.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1971.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1972.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1973.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1974.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1975.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1976.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1977.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1978.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Da

In [88]:
glob(local_data_folder + "/*.nc")

['/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1970.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1971.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1972.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1973.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1974.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1975.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1976.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1977.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1978.0.1x0.1.nc',
 '/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_1979.0.1x0.1.nc',
 '/Users/n

In [89]:
local_data_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test"

most_recent = glob(local_data_folder + "/*.nc")[-2]

# Explore .nc file
recent_data = netCDF4.Dataset(most_recent)

recent_data.variables

recent_data.variables['lat'][:]

len(recent_data.variables['lat'][:])

recent_data.variables['lon'][:]

len(recent_data.variables['lon'][:])

recent_data.variables['emi_bc']

recent_data.variables['emi_bc'][:, :]

recent_data.variables['emi_bc'].ncattrs()

repr(recent_data.variables['emi_bc'].getncattr('total_emi_bc'))

recent_data.ncattrs()

recent_data.dimensions

recent_data.dimensions['lat']

# recent_data.dimensions['lat'][:] =/ not subscriptable

len(recent_data.dimensions['lat'])

recent_data.variables['emi_bc'].dimensions

recent_data.variables['emi_bc'].size

# No time variable because all years are stored in separate files.
# Can use Rutger's function as a means of stacking them & adding the time dimension

6480000

In [90]:
most_recent[:-3] + "_edit.tif"

'/Users/nathansuberi/Desktop/RW_Data/Rasters/EDGAR/nc_test/v431_v2_REFERENCE_BC_2009.0.1x0.1_edit.tif'

In [102]:
# Can read to a geotiff with rasterio, or GDAL

# Vizzuality uses rasterio below

def netcdf2tif(dst,variable,outFile):
    nc = Dataset(dst)
    data = nc[variable][:,:]
    print(np.max(data))
            
#     data[data < 0] = -1
#     data[data == 32767.0] = -1
    
    print(data)
    
    # Return lat info
    south_lat = -90
    north_lat = 90

    # Return lon info
    west_lon = -180
    east_lon = 180
    
    rows, columns = data.shape              # get sizes

    # Reverse the array
#    flipped_array = np.fliplr(data) 

#     left_side = data[:,int(columns/2):]     # split the array... 
#     right_side = data[:,:int(columns/2)]    # ...into two halves. Then recombine.
#     wsg84_array = np.concatenate((left_side,right_side), axis=1)

    #reverse again
#    a = scipy.ndimage.interpolation.rotate(wsg84_array, 180)
#    fliped = np.fliplr(a)
    #plt.figure(figsize=(10,10))
    #plt.imshow(fliped, cmap=cm.jet)  
    
    print('transformation.......')
    # Transformation function
    transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    # Profile
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':np.float64, 
        'crs':'EPSG:4326', 
        'transform':transform, 
        'compress':'lzw'
    }
    with rio.open(outFile, 'w', **profile) as dst:
        dst.write(data.astype(profile['dtype']), 1)
    
    print('Data Shape: ',columns)
    print('Data Shape: ',rows)
    #os.remove('./'+file)

In [103]:
netcdf2tif(most_recent,"emi_bc", most_recent[:-3] + "_edit.tif")

1.3469e-08
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
transformation.......
Data Shape:  3600
Data Shape:  1800


In [92]:
with rio.open(most_recent[:-3] + "_edit.tif", 'r') as src:
    print(src.profile)
    data = src.read()

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -1.0, 'width': 3600, 'height': 1800, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.1, 0.0, 90.0, 0.0, -0.1), 'affine': Affine(0.1, 0.0, -180.0,
       0.0, -0.1, 90.0), 'compress': 'lzw', 'tiled': False, 'interleave': 'band'}


In [96]:
# Emissions gridmaps in ton substance / 0.1degree x 0.1degree / year for the .txt files 
# and in kg substance /m2 /s for the .nc files. 

np.max(data[0])

1.3469004578325894e-08

In [119]:
ftp_arctic = remote_path_north+arctic_file
ftp_antarctic = remote_path_south+antarctic_file
print(ftp_arctic)
print(ftp_antarctic)

local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/Polar_Sea_Ice/"
# Use 7: to not include the month's folder in the local file name
arctic_file_name = arctic_file[7:]
antarctic_file_name = antarctic_file[7:]

local_arctic = local_folder+arctic_file_name
local_antarctic = local_folder+antarctic_file_name

local_arctic_edit = local_arctic[:-4] + "_edit.tif"
local_antarctic_edit = local_antarctic[:-4] + "_edit.tif"

with closing(req.urlopen(ftp_arctic)) as r:
    with open(local_arctic, 'wb') as f:
        shutil.copyfileobj(r, f)
         
with closing(req.urlopen(ftp_antarctic)) as r:
    with open(local_antarctic, 'wb') as f:
        shutil.copyfileobj(r, f)

ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/08_Aug/N_201708_extent_v2.1.tif
ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/08_Aug/S_201708_extent_v2.1.tif


In [131]:
with rio.open(local_arctic) as src:
    pro = src.profile
    print(pro)
    data = src.read()
    
    pro.update(
        compress = "lzw",
        crs = "EPSG:4326"
    )
    
    with rio.open(local_arctic_edit, 'w', **pro) as dst:
        dst.write(data)
        
        
with rio.open(local_antarctic) as src:
    pro = src.profile
    print(pro)
    data = src.read()
    
    pro.update(
        compress = "lzw",
        crs = "EPSG:4326"
    )
    
    with rio.open(local_antarctic_edit, 'w', **pro) as dst:
        dst.write(data)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 304, 'height': 448, 'count': 1, 'crs': CRS({'init': 'epsg:3411'}), 'transform': (-3850000.0, 25000.0, 0.0, 5850000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3850000.0,
       0.0, -25000.0, 5850000.0), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 316, 'height': 332, 'count': 1, 'crs': CRS({'init': 'epsg:3412'}), 'transform': (-3950000.0, 25000.0, 0.0, 4350000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3950000.0,
       0.0, -25000.0, 4350000.0), 'tiled': False, 'interleave': 'band'}


/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [121]:
with rio.open(local_arctic_edit) as src:
    pro = src.profile
    print(pro)
    
with rio.open(local_antarctic_edit) as src:
    pro = src.profile
    print(pro)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 304, 'height': 448, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-3850000.0, 25000.0, 0.0, 5850000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3850000.0,
       0.0, -25000.0, 5850000.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 316, 'height': 332, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-3950000.0, 25000.0, 0.0, 4350000.0, 0.0, -25000.0), 'affine': Affine(25000.0, 0.0, -3950000.0,
       0.0, -25000.0, 4350000.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


In [127]:
### NOTE = THIS WILL KEEP REPLACING THE FILE WITH THE NEWEST VERSION

# S3 destinations
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/"

s3_file_antarctic = "cli_005_antarctic_sea_ice_extent.tif"
s3_key_antarctic_orig = s3_folder + s3_file_antarctic
s3_key_antarctic_edit = s3_key_antarctic_orig[:-4] + "_edit.tif"

s3_file_arctic = "cli_006_arctic_sea_ice_extent.tif"
s3_key_arctic_orig = s3_folder + s3_file_arctic
s3_key_arctic_edit = s3_key_arctic_orig[:-4] + "_edit.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [130]:
# Originals
s3_upload.upload_file(local_arctic, s3_bucket, s3_key_arctic_orig,
                         Callback=ProgressPercentage(local_arctic))

s3_upload.upload_file(local_antarctic, s3_bucket, s3_key_antarctic_orig,
                         Callback=ProgressPercentage(local_antarctic))

# Edits
s3_upload.upload_file(local_arctic_edit, s3_bucket, s3_key_arctic_edit,
                         Callback=ProgressPercentage(local_arctic_edit))

s3_upload.upload_file(local_antarctic_edit, s3_bucket, s3_key_antarctic_edit,
                         Callback=ProgressPercentage(local_antarctic_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/Polar_Sea_Ice/S_201708_extent_v2.1_edit.tif  5005 / 5005.0  (100.00%)

In [ ]:
# Experimentation

In [ ]:
# Convert .nc to .tif

# Code from Rutger Hofste

def netCDF4toGeotiff(fileName,fileLocation, lat_var, lon_var):
    netCDFInputBaseName = fileName.split('.')[0]
    nc_fid = Dataset(fileLocation, 'r')
    nc_attrs, nc_dims, nc_vars = ncdump(nc_fid, PRINT_METADATA)
    parameter = nc_vars[3]
    lats = nc_fid.variables[lat_var][:]  # extract/copy the data
    lons = nc_fid.variables[lon_var][:]
    times = nc_fid.variables['time'][:]
    timeUnit = nc_fid.variables["time"].getncattr("units")
    timeNormal =[]
    for time in times:
        if timeUnit == ("days since 1900-01-01 00:00:00") or (timeUnit =="Days since 1900-01-01"):
            timeNormal.append(datetime.datetime(1900,1,1) + datetime.timedelta(days=time))
        elif timeUnit == "days since 1901-01-01 00:00:00":
            timeNormal.append(datetime.datetime(1901,1,1) + datetime.timedelta(days=time))
        else:
            print "Error"
            timeNormal.append(-9999)
            
    for i in range(0,len(timeNormal)):
        #print timeNormal[i].year
        Z = nc_fid.variables[parameter][i, :, :]
        Z[Z<-9990]= -9999
        Z[Z>1e19] = -9999
        outputFilename = netCDFInputBaseName + "I%0.3dY%0.2dM%0.2d.tif" %(i,timeNormal[i].year,timeNormal[i].month)
        writefilename = os.path.join(EC2_OUTPUTPATH,outputFilename)
        writeFile(writefilename,geotransform,geoproj,Z)
    
    return time, timeUnit, timeNormal

def readFile(filename):
    filehandle = gdal.Open(filename)
    band1 = filehandle.GetRasterBand(1)
    geotransform = filehandle.GetGeoTransform()
    geoproj = filehandle.GetProjection()
    Z = band1.ReadAsArray()
    xsize = filehandle.RasterXSize
    ysize = filehandle.RasterYSize
    filehandle = None
    return xsize,ysize,geotransform,geoproj,Z

def writeFile(filename,geotransform,geoprojection,data):
    (x,y) = data.shape
    format = "GTiff"
    driver = gdal.GetDriverByName(format)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_datatype = gdal.GDT_Float32
    dst_ds = driver.Create(filename,y,x,1,dst_datatype, [ 'COMPRESS=LZW' ])
    dst_ds.GetRasterBand(1).SetNoDataValue(-9999)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds = None
    return 1

def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print "\t\ttype:", repr(nc_fid.variables[key].dtype)
            for ncattr in nc_fid.variables[key].ncattrs():
                print '\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print "\t\tWARNING: %s does not contain variable attributes" % key

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print "NetCDF Global Attributes:"
        for nc_attr in nc_attrs:
            print '\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print "NetCDF dimension information:"
        for dim in nc_dims:
            print "\tName:", dim
            print "\t\tsize:", len(nc_fid.dimensions[dim])
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print "NetCDF variable information:"
        for var in nc_vars:
            if var not in nc_dims:
                print '\tName:', var
                print "\t\tdimensions:", nc_fid.variables[var].dimensions
                print "\t\tsize:", nc_fid.variables[var].size
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars